In [5]:
from sklearn.model_selection import train_test_split

import tensorflow as tf
import numpy as np
import cv2
import glob
import pandas as pd

### Load and prepare data

In [6]:
n_classes = 8

images = [cv2.imread(file) for class_number in range(1,n_classes+1) for file in glob.glob('data/'+ str(class_number) + '/*.jpg')]
images = np.array(images)

In [7]:
Y = [[class_number] for class_number in range(1,n_classes+1) for file in glob.glob('data/'+ str(class_number) + '/*.jpg')]
Y = np.array(Y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(images, Y, test_size=0.2, random_state=42, stratify=Y)

In [9]:
X_train.shape, y_train.shape

((4480, 28, 28, 3), (4480, 1))

In [10]:
X_test.shape, y_test.shape

((1120, 28, 28, 3), (1120, 1))

In [11]:
y_train = pd.get_dummies(pd.DataFrame(y_train, columns=['label']), columns=['label']).values
y_test = pd.get_dummies(pd.DataFrame(y_test, columns=['label']), columns=['label']).values

In [12]:
y_train.shape, y_test.shape

((4480, 8), (1120, 8))

In [13]:
# convert in float and normalize to scale (0, 1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

### Load pretrained LeNet model

In [16]:
sess = tf.Session()

new_saver = tf.train.import_meta_graph('model/Classifier.meta')
new_saver.restore(sess, 'model/Classifier')

graph = tf.get_default_graph()
x = graph.get_tensor_by_name("x:0")
y = graph.get_tensor_by_name("y:0")
proba = graph.get_tensor_by_name("keep_proba:0")

feed_dict_train ={x: X_train, y: y_train, proba: 1.}
feed_dict_test ={x: X_test, y: y_test, proba: 1.}

accuracy_operation = graph.get_tensor_by_name("accur:0")
print('Train Accuracy by loaded model: {}'.format(sess.run(accuracy_operation, feed_dict_train)))
print('Test Accuracy by loaded model: {}'.format(sess.run(accuracy_operation, feed_dict_test)))

INFO:tensorflow:Restoring parameters from model/Classifier
Train Accuracy by loaded model: 0.9899553656578064
Test Accuracy by loaded model: 0.9866071343421936
